In [15]:
# USD Approximate Values of DG and RLY Liquidity Pools
capitalDG = 10000 
capitalRLY = 10000

# Estimated gas fee for the 4 transactions needed to claim, swap 1/2 of reward to Eth, add to LP, and stake LP token
fee = 30 * 4   #USD
# APY Adjustment modifier to the APY to quicky test the effects of different APYs on optimal compounding rate
adjust = 1
# APY 
rlyRate = 0.86 * adjust
dgRate = 0.4541 * adjust

#Interest Periods
time = 1 

# Boolean to flag a positive rate of change because idk how to find derivatives of iterated functions in Python
pos = False

In [16]:
# Compound function to determine portfolio value after compounding
def compound(capital,rate,time,fee,freq):
    for i in range (1,freq+1):
        capital = capital * (1 + (rate*time)) - fee
    return capital

days = []
i = 0
weeklyReturn = 0
# Loop through each Liquidity Pool
for rate in [rlyRate,dgRate]:
    if rate == rlyRate:
        # Identify Liquidity Pool and retrieve intial capital 
        pool = 'Rally'
        capital = capitalRLY
    elif rate == dgRate:
        # Identify Liquidity Pool and retrieve intial capital 
        pool = 'dg'
        capital = capitalDG
    diff = 0
    claim = 1
    # Compounding Frequencies to compare
    for freq in range(2,10,1):  
        # Calculate the portfolio size when compounded with a given frequency
        action = compound(capital,rate,time/freq,fee,freq) 
        # Calculate the portfolio size over the same time horizon, but claimed only once at the end.
        inaction = capital+capital * rate * time - fee
        # Subtract the two portfolio sizes. The frequency with the largest portfolio size will be saved and displayed
        if action - inaction > diff:
            diff = round(action - inaction)
            claim = freq
            reward = round(capital * rate * time/freq)
        print("freq: " + str(freq) + "    " + str(action - inaction))
    # Determine the number of days between each compounding event
    days.append(round(365/claim))
    # Determine the total weekly return of all Liquidity Pools
    weeklyReturn += round(reward*7/days[i])
    # Print optimal compounding frequency
    print('Max gains for ' +str(pool) +' at $' +str(diff) + ' and ' + str(days[i]) +' days per claim and frequency '+ str(claim))
    print('Generating $' + str(round(reward*7/days[i])) + ' per week')
    i +=1
# Print total weekly return of all Liquidity Pools
print('\n$' +str(weeklyReturn) + ' generated per week')


freq: 2    1677.3999999999978
freq: 3    2347.8482962962917
freq: 4    2654.2219012500027
freq: 5    2787.451924889596
freq: 6    2824.7168015030766
freq: 7    2803.2318760546004
freq: 8    2743.1637216024646
freq: 9    2656.3934236333153
Max gains for Rally at $2825 and 61 days per claim and frequency 6
Generating $164 per week
freq: 2    368.271025
freq: 3    424.795539726294
freq: 4    385.36081752149767
freq: 5    303.86198247329776
freq: 6    200.18831402482283
freq: 7    83.39486649808896
freq: 8    -41.80350948633895
freq: 9    -172.70971818342878
Max gains for dg at $425 and 122 days per claim and frequency 3
Generating $87 per week

$251 generated per week


In [17]:
# datetime functions to display number of days until the next compounding event
import datetime
today = datetime.date.today()
dgStart = datetime.date(2021,9,4)
rlyStart =datetime.date(2021,11,7)
rlyDays = today - rlyStart
dgDays = today - dgStart
print('rly staked ' + str(rlyDays.days) + ' days ago')
print('Claim in ' +str(days[0] - round(rlyDays.days,0)) + " days\n")
print('dg staked ' + str(dgDays.days) + ' days ago')
print('Claim in ' +str(days[1] - dgDays.days) +" days")

rly staked 5 days ago
Claim in 56 days

dg staked 69 days ago
Claim in 53 days


In [18]:
# Dictionary to store past APY values to find average APYs through the compounding period
APYrly = {'2021,11,7':0.8340
         }
